In [1]:
import pandas as pd
import json
import re

In [2]:
master_name = 'Eminem'
with open('../data/json_lyrics/Eminem.json','r') as f:
    data = json.load(f)

In [8]:
nodes = pd.DataFrame(columns=['node_name'])

In [3]:
ALL_ARTISTS = set()
ALL_ALBUMS = set()
ALL_SONGS = set()

In [4]:
def info_extract(art, album, songs):
    sep_name = re.sub('f./|f/|w/|&amp;|and|((^|\W)\()|(\)($|\W))', ',', art)
    list_of_names = sep_name.split(',')
    art_sng_rels = []
    alb_sng_rels = []
    global ALL_ALBUMS, ALL_ARTISTS, ALL_SONGS
    ALL_ALBUMS = ALL_ALBUMS|{album}
    for song in songs:
        song_name = song+'_'+album
        ALL_SONGS = ALL_SONGS|{song_name}
        alb_sng_rels.append({'album_node':album, 'song_node':song_name, 'typ':'contains_song'})
        for name in list_of_names:
            if 'raw_song_' in name:
                continue
            name = name.strip().lower()
            ALL_ARTISTS = ALL_ARTISTS|{name}
            art_sng_rels.append({'artist_node':name, 'song_node':song_name, 'typ':'raps_on'})
    return art_sng_rels, alb_sng_rels


In [5]:
art_sng_rels = []
alb_sng_rels = []
art_alb_rels = []
for name, albums in data.items():
    if type(albums) == str:
        albums = {albums:''}
    for album, songs in albums.items():
        if not (len(album) > 500 or 'www.' in album) and songs != '':
            art_alb_rels.append({'album_node':album, 'artist_node':master_name, 'typ':'primary_artist'})
        art_sngs, alb_sngs = info_extract(name, album, list(songs))
        art_sng_rels.extend(art_sngs)
        alb_sng_rels.extend(alb_sngs)

In [13]:
art_sng_df = pd.DataFrame(art_sng_rels)
alb_sng_df = pd.DataFrame(alb_sng_rels)
art_alb_df = pd.DataFrame(art_alb_rels)

In [14]:
art_sng_df

,artist_node,song_node,typ
0,eminem,Kill You_The Marshall Mathers LP,raps_on
1,eminem,Steve Berman_The Marshall Mathers LP,raps_on
2,eminem,Ken Kaniff_The Marshall Mathers LP,raps_on
3,eminem,Criminal_The Marshall Mathers LP,raps_on
4,eminem,Who Knew_The Marshall Mathers LP,raps_on
...,...,...,...
933,mudd,Slum Elementz_Searching for Jerry Garcia,raps_on
934,5 ela,Slum Elementz_Searching for Jerry Garcia,raps_on
935,t-3,Slum Elementz_Searching for Jerry Garcia,raps_on
936,slum village,Slum Elementz_Searching for Jerry Garcia,raps_on


In [15]:
alb_sng_df

,album_node,song_node,typ
0,The Marshall Mathers LP,Kill You_-6558703550866084594,contains_song
1,The Marshall Mathers LP,Steve Berman_-6558703550866084594,contains_song
2,The Marshall Mathers LP,Ken Kaniff_-6558703550866084594,contains_song
3,The Marshall Mathers LP,Criminal_-6558703550866084594,contains_song
4,The Marshall Mathers LP,Who Knew_-6558703550866084594,contains_song
...,...,...,...
472,Tupac Ressurection Soundtrack,One Day at a Time (Em's Version)_6824726271224...,contains_song
473,D12 World,American Psycho II_-51862449634763688,contains_song
474,Detox,I Need a Doctor_-4931677794823186217,contains_song
475,No.6 Collaborations Project,Remember the Name_-6900389319182977198,contains_song


In [28]:
art_alb_df.sort_values(by='album_node')

,album_node,artist_node,typ
269,"2.0 Boys 12""",Eminem,primary_artist
293,2011 BET Hip Hop Awards,Eminem,primary_artist
30,2017 BET Hip-Hop Awards,Eminem,primary_artist
20,8 Mile Soundtrack,Eminem,primary_artist
32,8 Mile Soundtrack/Curtain Call - The Hits,Eminem,primary_artist
...,...,...,...
217,West Kostra Nostra,Eminem,primary_artist
319,Westwood (BBC Radio 1),Eminem,primary_artist
231,"When to Stand Up 12""",Eminem,primary_artist
173,Wild Wild West Soundtrack,Eminem,primary_artist
